<a href="https://colab.research.google.com/github/jhuarancca/Applied-AI-Building-NLP-Apps-with-Hugging-Face-Transformers/blob/main/Code_01_XX_Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Please execute this notebook on the course python environment
#This installs all the pre-requisities for the course

!pip install tensorflow===2.10.0
!pip install torch
!pip install keras===2.10.0
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install evaluate
!pip install nltk
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 850.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [1]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

## 01.03. Using a Qu-An Pipeline

In [3]:
from transformers import pipeline

context="""
Earth is the third planet from the Sun and the only astronomical object
known to harbor life. While large volumes of water can be found
throughout the Solar System, only Earth sustains liquid surface water.
About 71% of Earth's surface is made up of the ocean, dwarfing
Earth's polar ice, lakes, and rivers. The remaining 29% of Earth's
surface is land, consisting of continents and islands.
Earth's surface layer is formed of several slowly moving tectonic plates,
interacting to produce mountain ranges, volcanoes, and earthquakes.
Earth's liquid outer core generates the magnetic field that shapes Earth's
magnetosphere, deflecting destructive solar winds.
"""
quan_pipeline = pipeline("question-answering",model="deepset/minilm-uncased-squad2")

answer=quan_pipeline(question="How much of earth is land?",context=context)
print(answer)

answer=quan_pipeline(question="How much of earth is ocean?",context=context)
print(answer)

answer=quan_pipeline(question="How much of earth is water?",context=context)
print(answer)

{'score': 0.9549782276153564, 'start': 323, 'end': 326, 'answer': '29%'}
{'score': 0.8307734727859497, 'start': 214, 'end': 217, 'answer': '91%'}
{'score': 0.14776480197906494, 'start': 101, 'end': 114, 'answer': 'large volumes'}


In [4]:
print("\nAnother question :")
print(quan_pipeline( question="How are mountain ranges created?",
             context=context))


Another question :
{'score': 0.2611975371837616, 'start': 393, 'end': 465, 'answer': "Earth's surface layer is formed of several slowly moving tectonic plates"}


In [6]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.0 MB/s eta 0:00:00


## 01.05 Evaluating Qu-An Performance

In [8]:
from evaluate import load
squad_metric = load("squad_v2")

#Ignoring Context & Question as they are not needed for evaluation
#This example is to showcase how the evaluation works based on match between the prediction
#and the correct answer

correct_answer="Paris"
predicted_answers=["Paris","London","Paris is one of the best cities in the world"]

cum_predictions=[]
cum_references=[]
for i in range(len(predicted_answers)):

    #Use the input format for predictions
    predictions = [{'prediction_text':predicted_answers[i],'id': str(i),'no_answer_probability': 0.}]
    cum_predictions.append(predictions[0])

    #Use the input format for naswers
    references = [{'answers': {'answer_start': [1],'text': [correct_answer]},'id': str(i)}]
    cum_references.append(references[0])

    results = squad_metric.compute(predictions=predictions,references=references)
    print("F1 is", results.get('f1'),
          " for answer :", predicted_answers[i])

#Compute for cumulative Results
cum_results=squad_metric.compute(predictions=cum_predictions,references=cum_references)
print("\n Cumulative Results : \n",cum_results)

F1 is 100.0  for answer : Paris
F1 is 0.0  for answer : London
F1 is 22.22222222222222  for answer : Paris is one of the best cities in the world

 Cumulative Results : 
 {'exact': 33.333333333333336, 'f1': 40.74074074074074, 'total': 3, 'HasAns_exact': 33.333333333333336, 'HasAns_f1': 40.74074074074074, 'HasAns_total': 3, 'best_exact': 33.333333333333336, 'best_exact_thresh': 0.0, 'best_f1': 40.74074074074074, 'best_f1_thresh': 0.0}
